In [1]:
import re

In [2]:
import numpy as np

In [3]:
src = r"""\begin{table}[H]
    \centering
    \begin{tabular}{ccccc} \toprule
        \multicolumn{1}{l}{} & \multicolumn{2}{c}{\textbf{Random Forest} ($MSE = 0.00117$)} & \multicolumn{2}{c}{\textbf{PADRE-RF} ($MSE = 0.156$)} \\ \cmidrule(r){2-5}
        \multicolumn{1}{c}{\textbf{emissions}} & \multicolumn{1}{c}{$\Delta MSE$} & top two $\Delta MSE$ & \multicolumn{1}{c}{$\Delta MSE$} & top two $\Delta MSE$ \\ \midrule
        \multicolumn{1}{c}{\multirow{2}{*}{\textbf{anthropogenic}}} & \multicolumn{1}{c}{$0.295$} & $0.0258$: $\text{CO}_2$ $_{(-12\text{h}, +16\text{l})}$ & \multicolumn{1}{c}{$0.22$} & $0.0264$: $\Delta$isoprene $_{(-48\text{h},+64\text{l})}$ \\
        \multicolumn{1}{c}{} & \multicolumn{1}{c}{(\#1)} & $0.0134$: pentanes $_{(-6\text{h},-8\text{l})}$ & \multicolumn{1}{c}{(\#2)} & $0.0154$: $\Delta \text{CO}_2$ $_{(-48\text{h},+64\text{l})}$ \\
        \multicolumn{1}{c}{\multirow{2}{*}{\textbf{temperature}}} & \multicolumn{1}{c}{$0.11$} & $0.0266$: temperature $_{(-48\text{h},\pm 2\text{l})}$ & \multicolumn{1}{c}{$0.355$} & $0.0887$: $\Delta$temperature $_{(-6\text{h},+8\text{l})}$ \\
        \multicolumn{1}{c}{} & \multicolumn{1}{c}{(\#2)} & $0.00645$: temperature $_{(-12\text{h},+16\text{l})}$ & \multicolumn{1}{c}{(\#1)} & $0.0706$: $\Delta$temperature $_{(-24\text{h},+32\text{l})}$ \\
        \multicolumn{1}{c}{\multirow{2}{*}{\textbf{anthr. $\text{NO}_{x}$}}} & \multicolumn{1}{c}{$0.0643$} & $0.0588$: $\text{NO}_{x}$ $_{(-12\text{h},+16\text{l})}$ & \multicolumn{1}{c}{$0.08$} & $0.0342$: $\text{NO}_{x}$ $_{(-48\text{h},+64\text{l})}$ \\
        \multicolumn{1}{c}{} & \multicolumn{1}{c}{(\#3)} & $0.00033$: $\text{NO}_{x}$ $_{(-6\text{h},-8\text{l})}$ & \multicolumn{1}{c}{(\#3)} & $0.00412$: $\Delta\text{NO}_{x}$ $_{(-48\text{h},+64\text{l})}$ \\
        \multicolumn{1}{c}{\multirow{2}{*}{\textbf{aerosols}}} & \multicolumn{1}{c}{$0.0147$} & $0.000965$: $400$-$1000\text{nm}$ $_{(-48\text{h},-64\text{l})}$ & \multicolumn{1}{c}{$0.0677$} & $0.0106$: $\Delta 50$-$70\text{nm}$ $_{(-48\text{h},+64\text{l})}$ \\
        \multicolumn{1}{c}{} & \multicolumn{1}{c}{(\#4)} & $0.000754$: $10$-$20\text{nm}$ $_{(-6\text{h},-8\text{l})}$ & \multicolumn{1}{c}{(\#4)} & $0.00167$: $\Delta 400$-$1000\text{nm}$ $_{(-48\text{h},+64\text{l})}$ \\
        \multicolumn{1}{c}{\multirow{2}{*}{\textbf{anthr. $\text{SO}_{2}$}}} & \multicolumn{1}{c}{$0.0102$} & $0.00776$: $\text{SO}_{2}$ $_{(-12\text{h},+16\text{l})}$ & \multicolumn{1}{c}{$0.0621$} & $0.00811$: $\text{SO}_{2}$ $_{(-12\text{h},+16\text{l})}$ \\
        \multicolumn{1}{c}{} & \multicolumn{1}{c}{(\#5)} & $0.000347$: $\text{SO}_{2}$ $_{(-6\text{h},-8\text{l})}$ & \multicolumn{1}{c}{(\#6)} & $0.00274$: $\Delta \text{SO}_{2}$ $_{(-48\text{h},+64\text{l})}$ \\
        \multicolumn{1}{c}{\multirow{2}{*}{\textbf{biogenic}}} & \multicolumn{1}{c}{$0.00314$} & $0.000167$: $\text{CH}_2\text{O}$ $_{(-48\text{h},\pm 2\text{l})}$ & \multicolumn{1}{c}{$0.0635$} & $0.0043$: MBO $_{(-24\text{h},-32\text{l})}$ \\
        \multicolumn{1}{c}{} & \multicolumn{1}{c}{(\#6)} & $0.00014$: aldehydes $_{(-48\text{h},-64\text{l})}$ & \multicolumn{1}{c}{(\#5)} & $0.00264$: $\Delta\text{CH}_{3}\text{CHO}$ $_{(-24\text{h},-32\text{l})}$ \\
        \multicolumn{1}{c}{\multirow{2}{*}{\textbf{monoterpenes}}} & \multicolumn{1}{c}{$0.00177$} & $0.000424$: $\alpha$-pinene $_{(-48\text{h},-64\text{l})}$ & \multicolumn{1}{c}{$0.0564$} & $0.000366$: $\alpha$-pinene $_{(-24\text{h},\pm 2\text{l})}$ \\
        \multicolumn{1}{c}{} & \multicolumn{1}{c}{(\#7)} & $0.000104$: $\beta$-pinene $_{(-48\text{h},-64\text{l})}$ & \multicolumn{1}{c}{(\#7)} & $0.000165$: $\Delta \beta$-pinene $_{(-1.5\text{h},+0\text{l})}$ \\
        \multicolumn{1}{c}{\multirow{2}{*}{\textbf{sesquiterpenes}}} & \multicolumn{1}{c}{$0.00144$} & $0.000171$: sqtrps $_{(-48\text{h},-64\text{l})}$ & \multicolumn{1}{c}{$0.0554$} & $0.000144$: $\Delta$sqtrps $_{(-6\text{h},\pm 1\text{l})}$ \\
        \multicolumn{1}{c}{} & \multicolumn{1}{c}{(\#8)} & $0.0000692$: sqtrps $_{(-48\text{h},\pm 2\text{l})}$ & \multicolumn{1}{c}{(\#8)} & $0.000124$: sqtrps $_{(-48\text{h},-64\text{l})}$ \\
        \multicolumn{1}{c}{\multirow{2}{*}{\textit{\textbf{ungrouped}}}} & \multicolumn{1}{c}{\multirow{2}{*}{--}} & $0.0203$: SSR $_{(-48\text{h},\pm 2\text{l})}$ & \multicolumn{1}{c}{\multirow{2}{*}{--}} & $0.0283$: $\Delta$DMS $_{(-24\text{h},-32\text{l})}$ \\
        \multicolumn{1}{c}{} & \multicolumn{1}{c}{} & $0.0113$: $\text{CH}_4$ $_{(-6\text{h},-8\text{l})}$ & \multicolumn{1}{c}{} & $0.0154$: $\Delta q_{met}$ $_{(-6\text{h},+8\text{l})}$ \\ \bottomrule
    \end{tabular}
    \caption[Permutation Feature Importance for Random Forests and PADRE-RF]{Permutation Feature Importance for Random Forests and PADRE-RF as measured by the increase in the mean-squared error. The permutations are performed both in the perturbations groups from \Cref{txt:perturbation-generalisation} and on a per-feature basis. For each group, the top two individual features are listed as well.}
    \label{tab:permutation-feature-importance}
\end{table}"""

In [4]:
def replace_float(m):
    f = np.round(float(m.group(0)), 4)
    
    return str(f)
    
#     e = int(np.floor(np.log10(f)))
#     r = f / 10**e
    
#     if e >= -3:
#         return str(f)
#     return fr"{int(np.round(r))} \cdot 10^{{{e}}}"

print(re.sub(r"0\.[0-9]+", replace_float, src))

\begin{table}[H]
    \centering
    \begin{tabular}{ccccc} \toprule
        \multicolumn{1}{l}{} & \multicolumn{2}{c}{\textbf{Random Forest} ($MSE = 0.0012$)} & \multicolumn{2}{c}{\textbf{PADRE-RF} ($MSE = 0.156$)} \\ \cmidrule(r){2-5}
        \multicolumn{1}{c}{\textbf{emissions}} & \multicolumn{1}{c}{$\Delta MSE$} & top two $\Delta MSE$ & \multicolumn{1}{c}{$\Delta MSE$} & top two $\Delta MSE$ \\ \midrule
        \multicolumn{1}{c}{\multirow{2}{*}{\textbf{anthropogenic}}} & \multicolumn{1}{c}{$0.295$} & $0.0258$: $\text{CO}_2$ $_{(-12\text{h}, +16\text{l})}$ & \multicolumn{1}{c}{$0.22$} & $0.0264$: $\Delta$isoprene $_{(-48\text{h},+64\text{l})}$ \\
        \multicolumn{1}{c}{} & \multicolumn{1}{c}{(\#1)} & $0.0134$: pentanes $_{(-6\text{h},-8\text{l})}$ & \multicolumn{1}{c}{(\#2)} & $0.0154$: $\Delta \text{CO}_2$ $_{(-48\text{h},+64\text{l})}$ \\
        \multicolumn{1}{c}{\multirow{2}{*}{\textbf{temperature}}} & \multicolumn{1}{c}{$0.11$} & $0.0266$: temperature $_{(-48\text{h},\pm 